In [ ]:
import numpy as np
import cv2
import imageio
import matplotlib.pyplot as plt
# importamos deepcopy para copiar objetos
from copy import deepcopy

In [ ]:
# cargamos las imagenes:
def load_images(filenames):
    return [imageio.imread(filename) for filename in filenames]

def find_centers(images):
    centers = [cv2.findCirclesGrid(image, (5, 4), None) for image in images]

    return centers

def show_centers(images, centers):
    # mostramos una matriz con las 23 imagenes y los centros encontrados
    images2 = deepcopy(images)
    tmp = [cv2.drawChessboardCorners(img, (5,4), cor[1], cor[0]) for img, cor in zip(images2, centers) if cor[0]]
    # now we show the images in a matrix, 4x6
    fig, ax = plt.subplots(4, 6, figsize=(20, 10))
    for i, image in enumerate(images2):
        ax[i//6, i%6].imshow(image)
        ax[i//6, i%6].axis('off')
        # ponemos el numero de la imagen
        ax[i//6, i%6].set_title(f'Image {i+1}')
    plt.show()
    return images2


def get_circle_centers(patron_shape, dx, dy):
    # patron_shape: (4, 6)
    # dx, dy: distancia entre centros
    points = []
    for i in range(patron_shape[1]):
        for j in range(patron_shape[0]):
            points.append([dx*i, dy*j])
    return np.array(points, dtype=np.float32)

def get_circle_centers2(chessboard_shape, dx, dy):
    return [[(i%chessboard_shape[0])*dx, (i//chessboard_shape[0])*dy, 0] for i in range(np.prod(chessboard_shape))]




In [ ]:
filenames = [f'calibracion/img{i}.jpg' for i in range(1,24)]
images = load_images(filenames)

/var/folders/11/8n3x035j1rngnz1l98ysm7vm0000gn/T/ipykernel_99632/767665108.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


In [ ]:
centers = find_centers(images)


In [ ]:
images2 = show_centers(images, centers)

In [ ]:
# centers:
valid_centers = [cor[1] for cor in centers if cor[0]]
num_valid_images = len(valid_centers)

# obtain their real points
real_points = get_circle_centers2((5, 4), 20, 20)
real_points = np.asarray([real_points for i in range(num_valid_images)], dtype=np.float32)

# We are going to convert our coordinates list in the reference system to numpy array
object_points = np.asarray([real_points for i in range(num_valid_images)], dtype=np.float32)

# convert the corners list to array
image_points = np.asarray(valid_centers, dtype=np.float32)

# now we calibrate the camera
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(real_points, image_points, images[0].shape[:2], None, None)

extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

print('Calibration matrix:')
print('-------------------')
print('Intrinsic parameters:')
print(intrinsics+'\n')
print('Distortion coefficients:')
print(dist_coeffs+'\n')
print('Extrinsic parameters:')
print(extrinsics+'\n')
print('RMS:')
print(rms+'\n')

error: OpenCV(4.8.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/calib3d/src/calibration.cpp:3408: error: (-210:Unsupported format or combination of formats) objectPoints should contain vector of vectors of points of type Point3f in function 'collectCalibrationData'


In [ ]:
if __name__ == '__main__':

    filenames = ['center/center_{:03d}.jpg'.format(i) for i in range(10)]

    images = load_images(filenames)

    centers, rets = find_centers(images)

    # visualize the centers
    images2 = show_centers(images, centers, rets)

    # obtain the centers of the circles
    real_points = get_circle_centers((4, 6), 1, 1)

    # now we calibrate the camera
    # we need to obtain the camera matrix and the distortion coefficients
    # we use the function cv2.calibrateCamera
    valid_centers = [centers[i] for i in range(len(rets)) if rets[i]]
    num_valid_images = len(valid_centers)
    # obtain their real points
    real_points = np.asarray([real_points for i in range(num_valid_images)], dtype=np.float32)

    # convert the corners list to array
    image_points = np.asarray(valid_centers, dtype=np.float32)

    # now we calibrate the camera
    rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(real_points, image_points, images[0].shape[:2], None, None)

    extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

    print('Calibration matrix:')
    print('-------------------')
    print('Intrinsic parameters:')
    print(intrinsics+'\n')
    print('Distortion coefficients:')
    print(dist_coeffs+'\n')
    print('Extrinsic parameters:')
    print(extrinsics+'\n')
    print('RMS:')
    print(rms+'\n')